In [1]:
ENV["COLUMNS"] = 1000

1000

In [3]:
###################* Uso de Procesos en cadena #############################
# TODO: Empleamos primero DataFrames + DataFramesMeta

using DataFrames, CSV, DataFramesMeta

df_ccpphuanuco = CSV.File("data/table_ccpp_inei.csv") |> DataFrame

first(df_ccpphuanuco, 10)

,fid,UBIGEO,CODCP,DEPARTAMEN,PROVINCIA,DISTRITO,CEN_POB,CAPITAL,POB,VIV,Y,X,FUENTE_FIN,Altitud
,Int64,Int64,Int64,String,String,String,String,String?,Int64,Int64,Float64,Float64,String,Int64
1,49060,100101,1001010001,HUANUCO,HUANUCO,HUANUCO,HUANUCO,Departamental,84612,24226,-9.93008,-76.2399,CENSO,1900
2,49061,100101,1001010002,HUANUCO,HUANUCO,HUANUCO,TRES DE MAYO DE YANACOCHA,missing,94,29,-9.83353,-76.3825,CENSO,3507
3,49062,100101,1001010003,HUANUCO,HUANUCO,HUANUCO,YURACYACU,missing,39,16,-9.90446,-76.3249,CENSO,2283
4,49063,100101,1001010004,HUANUCO,HUANUCO,HUANUCO,SAN ISIDRO DE CAPILLAPAMPA,missing,43,19,-9.8354,-76.3665,CENSO,2972
5,49064,100101,1001010005,HUANUCO,HUANUCO,HUANUCO,SANTO DOMINGO DE NAUYAN,missing,66,39,-9.90185,-76.2785,CENSO,2826
6,49065,100101,1001010006,HUANUCO,HUANUCO,HUANUCO,HUINISH,missing,23,11,-9.88892,-76.2871,CENSO,3315
7,49067,100101,1001010008,HUANUCO,HUANUCO,HUANUCO,QUEROCOCHA,missing,52,17,-9.88855,-76.2711,CENSO,3113
8,49069,100101,1001010010,HUANUCO,HUANUCO,HUANUCO,RIURA PAMPA,missing,21,13,-9.89919,-76.246,CENSO,2358
9,49071,100101,1001010012,HUANUCO,HUANUCO,HUANUCO,COLPA BAJA,missing,702,147,-9.8809,-76.2168,CENSO,1877


In [36]:
#* El Paquete "DataFramesMeta" nos proporciona el macro @chain desde Chain.jl (https://github.com/jkrumbiegel/Chain.jl) para unir más de un macro.

df_ccpphuanuco_group = @chain df_ccpphuanuco begin
    groupby(_, :DISTRITO)
    @combine(_, :Poblacion = sum(:POB), :Viviendas =sum(:VIV), :Ubigeo = unique(:UBIGEO), :MaxAlt = maximum(:Altitud))
    sort(:DISTRITO) #* El macro @orderby no funciona para datis agrupados solo para Dataframes
    @subset(_, :Poblacion .>= 5000, :Viviendas .> 2000, :MaxAlt .>=3000)
    @transform(_,
        Porc_pop = :Poblacion ./ sum(:Poblacion) *100,
        Porc_viv = :Viviendas ./ sum(:Viviendas) *100)
    insertcols!(_, 1, :Id => 1:nrow(_))
end

LoadError: syntax: missing comma or ) in argument list